In [1]:
from etils import epath
from ml_collections import config_dict
import numpy as np
import tensorflow as tf
import tqdm
import os

from chirp import audio_utils
from chirp.inference import embed_lib
from chirp.inference import tf_examples
import pandas as pd
import csv
import re

2024-06-09 15:56:35.627769: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-09 15:56:36.791256: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
# Define the model: Usually perch or birdnet.
model_choice = 'perch'  #@param
# Set the base directory for the project.
working_dir = '.'  #@param

# Set the embedding and labeled data directories.
embeddings_path = epath.Path(working_dir) / 'embeddings_logits'
labeled_data_path = epath.Path(working_dir) / 'labeled'
embeddings_glob = embeddings_path / 'embeddings-*'

# OPTIONAL: Set up separation model.
separation_model_key = 'separator_model_tf'  #@param
separation_model_path = ''  #@param

In [3]:
config = config_dict.ConfigDict()
config.embed_fn_config = config_dict.ConfigDict()
config.embed_fn_config.model_config = config_dict.ConfigDict()

# IMPORTANT: Select the target audio files.
# source_file_patterns should contain a list of globs of audio files, like:
# ['/home/me/*.wav', '/home/me/other/*.flac']
config.source_file_patterns = ['/home/mschulist/caples_sound/ARU_data/*/Data/*.wav']  #@param
config.output_dir = embeddings_path.as_posix()

# For Perch, set the perch_tfhub_model_version, and the model will load
# automagically from TFHub. Alternatively, set the model path for a local
# copy of the model.
# Note that only one of perch_model_path and perch_tfhub_version should be set.
perch_tfhub_version = 4  #@param
perch_model_path = ''  #@param

config.embed_fn_config.model_key = 'taxonomy_model_tf'
config.embed_fn_config.model_config.window_size_s = 5.0
config.embed_fn_config.model_config.hop_size_s = 5.0
config.embed_fn_config.model_config.sample_rate = 32000
config.embed_fn_config.model_config.tfhub_version = perch_tfhub_version
config.embed_fn_config.model_config.model_path = perch_model_path

bird_list = pd.read_csv('bird_list.csv')['code'].to_list()

# Only write embeddings to reduce size.
config.embed_fn_config.write_embeddings = True
config.embed_fn_config.write_logits = True
config.embed_fn_config.write_separated_audio = False
config.embed_fn_config.write_raw_audio = False

# Number of parent directories to include in the filename.
config.embed_fn_config.file_id_depth = 2

In [4]:
# Set up the embedding function, including loading models.
embed_fn = embed_lib.EmbedFn(**config.embed_fn_config)
print('\n\nLoading model(s)...')
embed_fn.setup()

# Create output directory and write the configuration.
output_dir = epath.Path(config.output_dir)
output_dir.mkdir(exist_ok=True, parents=True)
embed_lib.maybe_write_config(config, output_dir)

# config.embed_fn_config.shard_len_s = 60

# Create SourceInfos.
source_infos = embed_lib.create_source_infos(
    config.source_file_patterns,
    num_shards_per_file=config.get('num_shards_per_file', -1),
    shard_len_s=config.get('shard_len_s', -1))
print(f'Found {len(source_infos)} source infos.')

print('\n\nTest-run of model...')
window_size_s = config.embed_fn_config.model_config.window_size_s
sr = config.embed_fn_config.model_config.sample_rate
z = np.zeros([int(sr * window_size_s)])
embed_fn.embedding_model.embed(z)
print('Setup complete!')



Loading model(s)...


2024-06-09 15:56:40.991299: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Found 16036 source infos.


Test-run of model...


I0000 00:00:1717973805.311145 1542259 service.cc:145] XLA service 0x18ac8030 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1717973805.311203 1542259 service.cc:153]   StreamExecutor device (0): Host, Default Version
2024-06-09 15:56:45.714152: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
W0000 00:00:1717973805.722411 1542259 assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
I0000 00:00:1717973809.502530 1542259 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Setup complete!


In [5]:
# Calculate the subset of logit indicies to retain


# classes is a frozenset and as such is 'unordered' but in reality the tensor is in sorted order, we need the index
all_species = sorted(embed_fn.embedding_model.class_list['label'].classes)

with open('/home/mschulist/caples_sound/bird_list.csv', 'r') as f:
    caples_species = [line.rstrip('\n') for line in f]
    # strip header
    caples_species = sorted(caples_species[1:])

species_indicies = {}
for idx, bird in enumerate(all_species):
    for cbird in caples_species:
        if re.split('\d', bird)[0] == re.split('\d', cbird)[0]:
            species_indicies[idx] = cbird

with open(embeddings_path / 'logit_index.csv', 'w') as csv_file:  
    writer = csv.writer(csv_file)
    for k, v in species_indicies.items():
       writer.writerow([k, v, all_species[k]])

In [ ]:
min_file_size = 10_000_000

embed_fn.min_audio_s = 1.0
record_file = (output_dir / 'embeddings.tfrecord').as_posix()
succ, fail = 0, 0

existing_embedding_ids = embed_lib.get_existing_source_ids(
    output_dir, 'embeddings-*')

new_source_infos = embed_lib.get_new_source_infos(
    source_infos, existing_embedding_ids, config.embed_fn_config.file_id_depth)

print(f'Found {len(existing_embedding_ids)} existing embedding ids.'
      f'Processing {len(new_source_infos)} new source infos. ')

with tf_examples.EmbeddingsTFRecordMultiWriter(
    output_dir=output_dir, num_files=config.get('tf_record_shards', 1)) as file_writer:
    for source_info in tqdm.tqdm(new_source_infos, total=len(new_source_infos)):
        print(source_info, flush=True)
        file_id = source_info.filepath
        file_size = os.stat(file_id).st_size
        if file_size < min_file_size:
            fail += 1
            continue
        offset_s = source_info.shard_num * source_info.shard_len_s
        audio = audio_utils.load_audio_file(filepath=file_id, target_sample_rate=-1)
        # Unpack embed_fn.audio_to_example(file_id, offset_s, audio)
        # into 2 steps: embed(), model_outputs_to_tf_example()
        model_outputs = embed_fn.embedding_model.embed(audio)
        # prune the list of logits
        model_outputs.logits['label'] = model_outputs.logits['label'][:, list(species_indicies.keys())]
        print(model_outputs.logits['label'])
        example = tf_examples.model_outputs_to_tf_example(
            model_outputs=model_outputs,
            file_id=file_id,
            audio=audio,
            timestamp_offset_s=offset_s,
            write_raw_audio=embed_fn.write_raw_audio,
            write_separated_audio=embed_fn.write_separated_audio,
            write_embeddings=embed_fn.write_embeddings,
            write_logits=embed_fn.write_logits,
            write_frontend=embed_fn.write_frontend,
            tensor_dtype=embed_fn.tensor_dtype,
        )
        if example is None:
          fail += 1
          continue
        file_writer.write(example.SerializeToString())
        succ += 1
    file_writer.flush()

print(f'\n\nSuccessfully processed {succ} source_infos, failed {fail} times.')

2024-06-09 15:56:52.553310: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Found 0 existing embedding ids.Processing 16036 new source infos. 


  0%|                                                                  | 0/16036 [00:00<?, ?it/s]

SourceInfo(filepath='/home/mschulist/caples_sound/ARU_data/10-BLACK/Data/BLACK-10-CAP_20230608_003000.wav', shard_num=0, shard_len_s=-1.0)


W0000 00:00:1717973816.115024 1542259 assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
  0%|                                                      | 1/16036 [01:23<372:20:22, 83.59s/it]

...

[[-15.403352  -12.785922  -13.175775  ... -11.497602   -9.0150385
  -17.535305 ]
 [-14.918972  -13.234019  -12.770893  ... -11.436242   -8.19356
  -16.308666 ]
 [-13.987204  -12.485671  -12.424999  ... -11.227873   -8.145224
  -15.846727 ]
 ...
 [-15.5450945 -12.80419   -13.311833  ... -11.388999   -9.278044
  -16.302166 ]
 [-13.743786  -12.108593  -12.664417  ... -11.443954   -8.996488
  -16.884068 ]
 [-15.464741  -12.934417  -12.845484  ... -11.01406    -8.623228
  -17.000645 ]]
SourceInfo(filepath='/home/mschulist/caples_sound/ARU_data/26-LEIA/Data/26-LEIA_20230618_020000.wav', shard_num=0, shard_len_s=-1.0)


  5%|██▋                                              | 875/16036 [18:14:23<299:17:34, 71.07s/it]

[[-12.807906  -12.385941  -12.18001   ...  -9.967935   -8.323238
  -16.30016  ]
 [-14.5479765 -12.73709   -12.155912  ... -10.067306   -8.535432
  -16.569984 ]
 [-13.926598  -12.6146345 -12.90245   ...  -9.314315   -8.03121
  -15.989374 ]
 ...
 [-14.1049595 -12.231706  -12.567442  ... -10.074072   -8.236552
  -15.780415 ]
 [-13.606105  -11.935947  -11.727838  ... -10.037477   -8.302449
  -15.203031 ]
 [-13.088634  -12.432884  -11.533099  ...  -9.892847   -7.7173796
  -15.149115 ]]
SourceInfo(filepath='/home/mschulist/caples_sound/ARU_data/26-LEIA/Data/26-LEIA_20230616_070000.wav', shard_num=0, shard_len_s=-1.0)


  5%|██▋                                              | 876/16036 [18:15:32<296:31:10, 70.41s/it]

[[-14.194787  -13.749977  -13.884813  ... -13.77355    -8.968465
  -16.714396 ]
 [-14.039256  -10.731545  -15.443486  ... -10.190773   -9.392975
  -14.885126 ]
 [-12.5863905 -13.164433  -14.461879  ... -11.444743  -10.057643
  -15.666431 ]
 ...
 [-16.687954  -12.4519615 -14.58302   ... -10.889697   -9.441401
  -17.812187 ]
 [-15.747779  -11.315026  -14.294992  ... -10.113961   -9.248112
  -17.224007 ]
 [-16.505936  -11.920996  -15.156046  ... -11.557052   -9.875872
  -16.796495 ]]
SourceInfo(filepath='/home/mschulist/caples_sound/ARU_data/26-LEIA/Data/26-LEIA_20230607_093000.wav', shard_num=0, shard_len_s=-1.0)


# OLD STUFF BELOW!!

In [ ]:
min_file_size = 10_000_000

embed_fn.min_audio_s = 1.0
record_file = (output_dir / 'embeddings.tfrecord').as_posix()
succ, fail = 0, 0

existing_embedding_ids = embed_lib.get_existing_source_ids(
    output_dir, 'embeddings-*')

new_source_infos = embed_lib.get_new_source_infos(
    source_infos, existing_embedding_ids, config.embed_fn_config.file_id_depth)

print(f'Found {len(new_source_infos)} existing embedding ids.'
      f'Processing {len(new_source_infos)} new source infos. ')

with tf_examples.EmbeddingsTFRecordMultiWriter(
    output_dir=output_dir, num_files=config.get('tf_record_shards', 1)) as file_writer:
    for source_info in new_source_infos:
        print(source_info, flush=True)
        file_id = source_info.filepath
        file_size = os.stat(file_id).st_size
        if file_size < min_file_size:
            fail += 1
            continue
        offset_s = source_info.shard_num * source_info.shard_len_s
        audio = audio_utils.load_audio_file(filepath=file_id, target_sample_rate=-1)
        example = embed_fn.audio_to_example(file_id, offset_s, audio)
        # example = None
        if example is None:
          fail += 1
          continue
        file_writer.write(example.SerializeToString())
        succ += 1
    file_writer.flush()

print(f'\n\nSuccessfully processed {succ} source_infos, failed {fail} times.')

In [6]:
new_source_infos[0]

SourceInfo(filepath='ARU_data/10-BLACK/Data/BLACK-10-CAP_20230608_003000.wav', shard_num=0, shard_len_s=-1.0)

In [ ]:
print(os.stat(new_source_infos[0].filepath))
print(os.stat(new_source_infos[1].filepath))

In [8]:
fns = [fn for fn in epath.Path('./embeddings').glob('embeddings-*')]
ds = tf.data.TFRecordDataset(fns)
parser = tf_examples.get_example_parser(['label'])
ds = ds.map(parser)
for ex in ds.as_numpy_iterator():
  print(ex['filename'])
  print(ex['embedding'].shape, flush=True)
  break

b'10-BLACK/Data/BLACK-10-CAP_20230608_003000.wav'
(179, 1, 1280)


In [5]:
#@title Run embedding. { vertical-output: true }

# Uses multiple threads to load audio before embedding.
# This tends to be faster, but can fail if any audio files are corrupt.

min_file_size = 10_000_000



embed_fn.min_audio_s = 1.0
record_file = (output_dir / 'embeddings.tfrecord').as_posix()
succ, fail = 0, 0

existing_embedding_ids = embed_lib.get_existing_source_ids(
    output_dir, 'embeddings-*')

new_source_infos = embed_lib.get_new_source_infos(
    source_infos, existing_embedding_ids, config.embed_fn_config.file_id_depth)

filtered_source_infos = []

for s in new_source_infos:
    size = os.stat(s.filepath).st_size
    if size < min_file_size:
        continue
    filtered_source_infos.append(s)

new_source_infos = filtered_source_infos

print(f'Found {len(new_source_infos)} existing embedding ids.'
      f'Processing {len(new_source_infos)} new source infos. ')

try:
  audio_loader = lambda fp, offset: audio_utils.load_audio_window(
      fp, offset, sample_rate=config.embed_fn_config.model_config.sample_rate,
      window_size_s=config.get('shard_len_s', -1.0))
  audio_iterator = audio_utils.multi_load_audio_window(
      filepaths=[s.filepath for s in new_source_infos],
      offsets=[s.shard_num * s.shard_len_s for s in new_source_infos],
      audio_loader=audio_loader,
  )
  with tf_examples.EmbeddingsTFRecordMultiWriter(
      output_dir=output_dir, num_files=config.get('tf_record_shards', 1)) as file_writer:
    for source_info, audio in tqdm.tqdm(
        zip(new_source_infos, audio_iterator), total=len(new_source_infos)):
      file_id = source_info.file_id(config.embed_fn_config.file_id_depth)
      offset_s = source_info.shard_num * source_info.shard_len_s
      example = embed_fn.audio_to_example(file_id, offset_s, audio)
      if example is None:
        fail += 1
        continue
      file_writer.write(example.SerializeToString())
      succ += 1
    file_writer.flush()
finally:
  del(audio_iterator)
print(f'\n\nSuccessfully processed {succ} source_infos, failed {fail} times.')

fns = [fn for fn in output_dir.glob('embeddings-*')]
ds = tf.data.TFRecordDataset(fns)
parser = tf_examples.get_example_parser()
ds = ds.map(parser)
for ex in ds.as_numpy_iterator():
  print(ex['filename'])
  print(ex['embedding'].shape, flush=True)
  break

Found 15759 existing embedding ids.Processing 15759 new source infos. 


  0%|                                                 | 0/15759 [00:00<?, ?it/s]2024-05-21 11:53:26.968554: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:38] Ignoring Assert operator jax2tf_infer_fn_/assert_equal_1/Assert/AssertGuard/Assert
2024-05-21 11:53:30.890136: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng0{} for conv (f32[179,640,501,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[179,1,160640,1]{3,2,1,0}, f32[640,1,640,1]{3,2,1,0}), window={size=640x1 stride=320x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"conv_result_scale":1,"activation_mode":"kNone","side_input_scale":0,"leakyrelu_alpha":0} is taking a while...
2024-05-21 11:53:31.866393: E external/local_xla/xla/service/slow_operation_alarm.cc:133] The operation took 1.976444696s
Trying algorithm eng0{} for conv (f32[179,640,501,1]{3,2,1,0}, u8[0]{0}) custom-call(f32[179,1,160640,1]{3,2,1,0}, f32[640,1,640,1]{3,2,1,0}), window={size=640x1 stride=3

KeyboardInterrupt: 

In [5]:
# Uses multiple threads to load audio before embedding.
# This tends to be faster, but can fail if any audio files are corrupt.

min_file_size = 10_000_000

embed_fn.min_audio_s = 1.0
record_file = (output_dir / 'embeddings.tfrecord').as_posix()
succ, fail = 0, 0

existing_embedding_ids = embed_lib.get_existing_source_ids(
    output_dir, 'embeddings-*')

new_source_infos = embed_lib.get_new_source_infos(
    source_infos, existing_embedding_ids, config.embed_fn_config.file_id_depth)

print(f'Found {len(new_source_infos)} existing embedding ids.'
      f'Processing {len(new_source_infos)} new source infos. ')

filtered_source_infos = []

for s in new_source_infos:
    size = os.stat(s.filepath).st_size
    if size < min_file_size:
        continue
    filtered_source_infos.append(s)

new_source_infos = filtered_source_infos


audio_iterator = audio_utils.multi_load_audio_window(
    filepaths=[s.filepath for s in new_source_infos],
    offsets=[s.shard_num * s.shard_len_s for s in new_source_infos],
    sample_rate=config.embed_fn_config.model_config.sample_rate,
    window_size_s=config.get('shard_len_s', -1.0),
)

with tf_examples.EmbeddingsTFRecordMultiWriter(
    output_dir=output_dir, num_files=config.get('tf_record_shards', 1)) as file_writer:
  for source_info, audio in tqdm.tqdm(
      zip(new_source_infos, audio_iterator), total=len(new_source_infos)):
    file_id = source_info.file_id(config.embed_fn_config.file_id_depth)
    file_size = os.stat(source_info.filepath).st_size
    if file_size < min_file_size:
        fail += 1
        continue
    offset_s = source_info.shard_num * source_info.shard_len_s
    example = embed_fn.audio_to_example(file_id, offset_s, audio)
    # example = None
    if example is None:
      fail += 1
      continue
    file_writer.write(example.SerializeToString())
    succ += 1
  file_writer.flush()
print(f'\n\nSuccessfully processed {succ} source_infos, failed {fail} times.')

fns = [fn for fn in output_dir.glob('embeddings-*')]
ds = tf.data.TFRecordDataset(fns)
parser = tf_examples.get_example_parser()
ds = ds.map(parser)
for ex in ds.as_numpy_iterator():
  print(ex['filename'])
  print(ex['embedding'].shape, flush=True)
  break

Found 14949 existing embedding ids.Processing 14949 new source infos. 
SourceInfo(filepath='/home/mschulist/caples_sound/ARU_data/10-BLACK/Data/BLACK-10-CAP_20230608_003000.wav', shard_num=0, shard_len_s=-1.0)


TypeError: multi_load_audio_window() got an unexpected keyword argument 'sample_rate'